# Eurovision Song Contest Data

This notebook imports all the data required to visualise and model trends in the Eurovision voting.
The data imported is from the following sources:

- Voting Scores
    - 1975-2019 data from [Kaggle](https://www.kaggle.com/datasets/datagraver/eurovision-song-contest-scores-19752019)
    - 2020 was cancelled
    - 2021, 2022 scraped from Wikipedia
    - We limit to 1998 onwards because this is when the voting changed to include tele-voting.
- Song and country languages
    - Performance language from [Kaggle](https://www.kaggle.com/datasets/minitree/eurovision-song-lyrics?select=eurovision-lyrics-2022.json)
    - Official country language from [wikipedia](https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory)
- Gender
    - The gender data is guessed by scraping the wikipedia page for the performing artist
- Migration data between performing and voting countries
    - [Our World in Data](https://ourworldindata.org/migration) on international migration, under the 'Explore data on where people migrate from and to' section.
       Original source is from the UN. Data shows total number of immigrants in each country split by country of origin in the years 1990-2020, recorded at intervals of every 5 years.
    - Country populations are from the [World Bank](https://data.worldbank.org/indicator/SP.POP.TOTL?end=2021&start=2021&view=map).
- Country borders
    - [GeoDataSource](https://github.com/geodatasource/country-borders/)   
- Competition winners
    - [Wikipedia](https://en.wikipedia.org/wiki/List_of_Eurovision_Song_Contest_winners)
- 2023 performers
    - [Wikipedia](https://en.wikipedia.org/wiki/Eurovision_Song_Contest_2023)

# Voting scores

We import the voting scores from a variety of sources: 
 - 1975-2019 data from Kaggle: https://www.kaggle.com/datasets/datagraver/eurovision-song-contest-scores-19752019
 - 2020 was cancelled
 - 2021, 2022 scraped from Wikipedia

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import re
import pycountry
import json
import aiohttp
import asyncio
import requests
from bs4 import BeautifulSoup

In [ ]:
## Read in data from the kaggle dataset
votes_1975_2019 = pd.read_excel("data/eurovision_song_contest_1975_2019.xlsx")

# Clean up column names first
votes_1975_2019.columns = [c.strip().lower().replace(' ', '_') for c in votes_1975_2019.columns.values.tolist()]

print(votes_1975_2019.shape)
votes_1975_2019.head()

In [ ]:
## Clean up this dataset.

# Select only finals votes, and only 1998 onwards (inclusive)
votes_1998_2019 = votes_1975_2019[(votes_1975_2019['(semi-)_final'] == 'f') & (votes_1975_2019['year'] >= 1998)]

# Drop unnecessary columns
votes_1998_2019 = votes_1998_2019[["year", "from_country", "to_country", "points", "jury_or_televoting"]]

# Clean up country names
def standardise_country(c):
    replacements = [('-', ' '), ('&', 'and'), ('netherands', 'netherlands'),
                    # FYR Macedonia was formally renamed as North Macedonia in 2019
                    ('f.y.r. macedonia', 'north macedonia'), 
                    ('russia', 'russian federation'), 
                    ('the netherlands', 'netherlands'), 
                    ('czech republic', 'czechia'),
                    # Yugoslavia dissolved in 2002; most of it became 'Serbia and Montenegro', until 2006, when Serbia and Montenegro split ways.
                    ('serbia and montenegro', 'yugoslavia'),
                    ('moldova', 'moldova, republic of')]
    c = c.lower()
    for r in replacements:
        c = c.replace(r[0], r[1])
    return c

for column in ['from_country', 'to_country']:
    votes_1998_2019[column] = votes_1998_2019[column].map(standardise_country)

# Drop columns which correspond to the same vote (there are two Belarus -> Russia in 2019, for example)
votes_1998_2019 = votes_1998_2019.drop_duplicates(subset=['year', 'from_country', 'to_country', 'jury_or_televoting'])

# Drop Lithuania in 2003 (they didn't participate - I don't know why it's still in the dataset)
votes_1998_2019 = votes_1998_2019[~((votes_1998_2019['to_country'] == 'lithuania') & (votes_1998_2019['year'] == 2003))]

# Drop "votes" from one country to itself
votes_1998_2019 = votes_1998_2019[votes_1998_2019['from_country'] != votes_1998_2019['to_country']]

votes_1998_2019.sample(n=10)


In [ ]:
## Now we need to fetch some data from Wikipedia for the 2021 and 2022 contests.

import requests
from bs4 import BeautifulSoup

def import_votes_from_wp(year: int) -> pd.DataFrame:
    # ID numbers for the respective tables on the Wikipedia page.
    JURY_ID = 16
    TELEVOTING_ID = 17

    url = f"https://en.wikipedia.org/wiki/Eurovision_Song_Contest_{year}#Final_2"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('table', {'class': "wikitable"})

    def parse_table_from_id(wp_id: int, jury_or_tele: str) -> pd.DataFrame:
        df_table = pd.read_html(str(tables[wp_id]))
        df_table = pd.DataFrame(df_table[0])

        # remove redundant rows/columns
        df_table = df_table.drop(df_table.columns[[0, 2, 3, 4]], axis=1)
        df_table = df_table.drop(df_table.index[[0, 2]], axis=0) 

        # set the index to the first column
        df_table = df_table.set_index(df_table.columns[0])

        # set the column names as the first row
        df_table.columns = df_table.iloc[0]
        df_table = df_table.drop(df_table.index[0])

        # replace NaN with 0
        df_table = df_table.fillna(0)

        # squash the column index with stack
        df_table = df_table.stack().reset_index()

        df_table.columns = ['to_country', 'from_country', 'points']
        df_table['jury_or_televoting'] = jury_or_tele

        df_table['year'] = year

        # re-order the columns to match the original data   
        df_table = df_table[['year', 'from_country', 'to_country', 'points', 'jury_or_televoting']]
        
        df_table['points'] = df_table['points'].astype(int)
        
        # Clean up countries as before
        for column in ['from_country', 'to_country']:
            df_table[column] = df_table[column].map(standardise_country)

        return(df_table)

    jury_table = parse_table_from_id(JURY_ID, jury_or_tele='J')
    tele_table = parse_table_from_id(TELEVOTING_ID, jury_or_tele='T')
    return(pd.concat([jury_table, tele_table]))

votes_1998_2022 = pd.concat([votes_1998_2019,
                             import_votes_from_wp(2021),
                             import_votes_from_wp(2022)])

# Again, drop "votes" from one country to herself
votes_1998_2022 = votes_1998_2022[votes_1998_2022['from_country'] != votes_1998_2022['to_country']]

votes_1998_2022.sample(n=10)

In [ ]:
# This cell is a sanity check to make sure that all countries participating in 
# a given year got the same number of votes. We hope to see the 'is_consistent' 
# column be True for all years in the output.

def check_consistency(df):
    def all_entries_same(arr : np.ndarray) -> bool:
        # Determines if all non-NaN entries in a numpy array have the same value.
        arr2 = arr[~np.isnan(arr)]
        return np.all(arr2 == arr2[0])

    # Pivot to wide form, so that each row gives the number of scores each country received in a given year
    grouped_votes = df.groupby(by=['year', 'to_country'])['points'].count().reset_index()
    grouped_votes = grouped_votes.pivot(index="year", columns="to_country", values="points")
    # Create "is_consistent" column and move it to the front
    col_names = grouped_votes.columns
    grouped_votes["is_consistent"] = grouped_votes.apply(all_entries_same, axis=1, raw=True)
    new_col_names = ["is_consistent", *col_names]

    if(not all(grouped_votes["is_consistent"])):     
        # Show data
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(grouped_votes.reindex(columns=new_col_names))

        raise Exception("Inconsistent number of votes received by countries in some years!")
    else: 
        print("All years are consistent!")
        
check_consistency(votes_1998_2022)

In [ ]:
# Now we need to combine jury and televoting scores.

# Years where jury voting happened
jury_years = np.unique(votes_1998_2022[votes_1998_2022['jury_or_televoting'] == 'J']['year'])
# Years where televoting happened
televoting_years = np.unique(votes_1998_2022[votes_1998_2022['jury_or_televoting'] == 'T']['year'])
# Years where both happened (i.e. the intersection)
double_voting_years = np.intersect1d(jury_years, televoting_years)

In [ ]:
# These are the years for which the points can just be used as-is.
votes_to_keep = votes_1998_2022[~votes_1998_2022['year'].isin(double_voting_years)]
votes_to_keep = votes_to_keep.drop(columns=['jury_or_televoting'])

# These are the years which we need to process.
# The way we do this is to add up the J and T scores, then re-rank them and assign 12 points to the highest score, 10 to the next-highest, etc.
votes_to_process = votes_1998_2022[votes_1998_2022['year'].isin(double_voting_years)]
summed_votes = votes_to_process.sort_values(by=['year', 'from_country', 'to_country'])
summed_votes = summed_votes.groupby(by=['year', 'from_country', 'to_country']).sum(numeric_only=True)

def rescale_points(pts: pd.Series) -> pd.Series:
    # grp is a pd.Series corresponding to one combination of 'year' and 'from_country'
    ranks_to_rescaled_points = {1: 12, 2: 10, 3: 8, 4: 7, 5: 6, 6: 5, 7: 4, 8: 3, 9: 2, 10: 1}
    ranks = [sorted(pts, reverse=True).index(pt) + 1 for pt in pts]
    rescaled_points = {pt: ranks_to_rescaled_points.get(r, 0) for pt, r in zip(pts, ranks)}
    return pts.map(rescaled_points)

processed_votes = summed_votes.groupby(by=['year', 'from_country']).transform(rescale_points).reset_index()
processed_votes.head()

In [ ]:
# Sanity check
x = processed_votes.rename(columns={"points": "rescaled"})
x = x.set_index(["year", "from_country", "to_country"])
v = summed_votes.reset_index().set_index(["year", "from_country", "to_country"])
joined = v.join(x, how="outer").reset_index()
joined[(joined['year'] == 2016) & (joined['from_country'] == 'albania')].sort_values(by="points", ascending=False)

In [ ]:
# Add in country codes, and that's our final voting data.
votes = pd.concat([votes_to_keep, processed_votes]).reset_index(drop=True)

def get_country_codes(name):
    if name == 'yugoslavia':
        # That's how it's encoded in pycountry.
        # https://github.com/flyingcircusio/pycountry/blob/main/src/pycountry/databases/iso3166-3.json
        cty = pycountry.historic_countries.get(name='yugoslavia, socialist federal republic of')
    else:
        cty = pycountry.countries.get(name=name)
    if cty is None:
        raise KeyError("Country name " + name + " not found in pycountry. This really shouldn't happen.")
    
    return cty.alpha_2, cty.alpha_3

for ft in ['from', 'to']:
    votes[f'{ft}_code2'], votes[f'{ft}_code3'] = zip(*votes[f'{ft}_country'].map(get_country_codes))

# Add column for each country and year get the total number of points received
votes['total_points'] = votes.groupby(by=['year', 'to_country'])['points'].transform('sum')

# For each year rank the countries by total points received, where draws get same value
temp = votes[['year', 'to_country', 'total_points']].drop_duplicates()
temp['rank'] = temp.groupby(by=['year'])['total_points'].rank(method='first', ascending=False)

# merge votes with ranks
votes = votes.merge(temp, on=['year', 'to_country', 'total_points'], how='left')


votes

In [ ]:
# sanity check the numbers
temp = votes[['from_country', 'year']].value_counts()

# for each year print the unique values
for year, group in temp.groupby(level=1):
    print(year, group.unique())

In [ ]:
check_consistency(votes)

In [ ]:
votes.to_csv('data/votes.csv', index=False)

# Song language

- Performance language from Kaggle: https://www.kaggle.com/datasets/minitree/eurovision-song-lyrics?select=eurovision-lyrics-2022.json
- Official country language from wikipedia: https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory

In [ ]:
songs = pd.read_json('data/eurovision-lyrics-2022.json').T
songs = songs[['Country', 'Artist', 'Language', 'Year']]

# Rename a couple of columns
songs = songs.rename(columns={'Language': 'Language_sung'})

# Tidy up country names
for original, replacement in [('Macedonia', 'North Macedonia'),
                              ('Russia', 'russian federation'),
                              ('Serbia and Montenegro', 'yugoslavia'),
                              ('Moldova', 'moldova, republic of'),
                              ('Czech Republic', 'czechia'),
                              ('The Netherlands', 'netherlands')]:
    songs.loc[songs['Country'] == original, 'Country'] = replacement
songs['Country'] = songs['Country'].str.lower()

# Limit to 1998 and later
songs['Year'] = pd.to_numeric(songs['Year'])
songs = songs[songs['Year'] > 1997]

# Add country code columns
songs['Country_code2'], songs['Country_code3'] = zip(
    *songs['Country'].map(get_country_codes))

songs.head()

In [ ]:
# Tidy up the language sung column
songs['Language_sung'] = songs['Language_sung'].str.lower()
songs['Language_sung'] = songs['Language_sung'].str.replace('partly|dialect|title|and', '', regex=True)

# for each key in the dictionary, replace the value with the key
replace_strings = {
    'fr\\.': 'french', 'eng\\.': 'english', 'gr\\.': 'greek', 
    'sp\\.': 'spanish', 'rom\\.': 'romanian', 'russ\\.': 'russian',
    'it\\.': 'italian', 'germ\\.': 'german', 'pol\\.': 'polish', 
    'sign language': 'sign-language'
}

for key, value in replace_strings.items():
    songs['Language_sung'] = songs['Language_sung'].str.replace(key, value, regex=True)

def extract_languages(lang_string):
    """Convert the string in language_sung into a list of languages"""
    langs = re.split(r'\s*[/()]\s*', lang_string)
    langs = [lang.strip() for lang in langs]
    return [lang for lang in langs if lang != ""]

songs['Language_sung'] = songs['Language_sung'].apply(extract_languages)
songs.tail(n=10)

In [ ]:
songs['Contains_English'] = songs['Language_sung'].apply(lambda x: 'english' in x)
songs['Contains_NonEnglish'] = songs['Language_sung'].apply(lambda x: x != ['english'])

songs[['Contains_English', 'Contains_NonEnglish']].value_counts()

In [ ]:
print('Number of songs containing English or non-English:')

songs[['Contains_English', 'Contains_NonEnglish']].value_counts()

Next we want to see whether countries are singing in their official language. We can get the official language from [Wikipedia](https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory).

In [ ]:
# Get the official languages from Wikipedia

url = (
    f"https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory"
)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
tables = soup.find_all("table", {"class": "wikitable"})

table = tables[0]
df_languages = pd.read_html(str(table))
df_languages = pd.DataFrame(df_languages[0])

# Tidy the columns
df_languages = df_languages.fillna("")
df_languages["Country/Region"] = df_languages["Country/Region"].apply(
    lambda x: re.sub("\[.*?\]", "", x)
)
df_languages.rename(columns={"Official language": "Official_languages"}, inplace=True)
df_languages["Official_languages"] = df_languages["Official_languages"].apply(
    lambda x: re.sub("\[.*?\]", "", x)
)

# Tidy the country names
df_languages["Country/Region"] = df_languages["Country/Region"].str.lower()

df_languages.loc[
    df_languages["Country/Region"] == "united kingdom and crown dependencies etc.",
    "Country/Region",
] = "united kingdom"
df_languages.loc[
    df_languages["Country/Region"] == "russia", "Country/Region"
] = "russian federation"
df_languages.loc[
    df_languages["Country/Region"] == "serbia and montenegro", "Country/Region"
] = "yugoslavia"
df_languages.loc[
    df_languages["Country/Region"] == "moldova", "Country/Region"
] = "moldova, republic of"
df_languages.loc[
    df_languages["Country/Region"] == "czech republic", "Country/Region"
] = "czechia"
df_languages = pd.concat(
    [
        df_languages,
        pd.Series(
            {
                "Country/Region": "yugoslavia",
                "Official_languages": "serbian montenegrin",
            }
        )
        .to_frame()
        .T,
    ],
    ignore_index=True,
)
set(songs["Country"].unique()) - set(df_languages["Country/Region"].unique())

df_languages = df_languages.loc[df_languages["Country/Region"].isin(songs["Country"].unique())]

df_languages["Country_code2"], df_languages["Country_code3"] = zip(
    *df_languages["Country/Region"].map(get_country_codes)
)

# Tidy the language column
df_languages["Official_languages"] = df_languages["Official_languages"].str.lower()
df_languages["Official_languages"] = df_languages["Official_languages"].apply(
    lambda x: x.replace("all have de facto status", "")
)
df_languages["Official_languages"] = df_languages["Official_languages"].apply(
    lambda x: x.replace(",", "")
)
df_languages["Official_languages"] = df_languages["Official_languages"].apply(
    lambda x: x.replace("(", "")
)
df_languages["Official_languages"] = df_languages["Official_languages"].apply(
    lambda x: x.replace(")", "")
)

# Manually add missing languages
df_languages.loc[df_languages["Country_code2"] == "LT", "Official_languages"] = (
    "samogitian "
    + df_languages.loc[df_languages["Country_code2"] == "LT", "Official_languages"]
)
df_languages.loc[df_languages["Country_code2"] == "FR", "Official_languages"] = (
    "breton corsican "
    + df_languages.loc[df_languages["Country_code2"] == "FR", "Official_languages"]
)
df_languages.loc[df_languages["Country_code2"] == "SI", "Official_languages"] = (
    "slovenian " + df_languages.loc[df_languages["Country_code2"] == "SI", "Official_languages"]
)
df_languages.loc[df_languages["Country_code2"] == "EE", "Official_languages"] = (
    "võro " + df_languages.loc[df_languages["Country_code2"] == "EE", "Official_languages"]
)

df_languages.tail()

In [ ]:
songs.head()

In [ ]:
# print any countries in songs['Country'] that are not in df_languages['Country/Region']
if len(set(songs['Country_code2']) - set(df_languages['Country_code2'])) > 0: 
    countries = list(set(songs['Country_code2']) - set(df_languages['Country_code2']))
    raise KeyError("Country name " + ', '.join(countries) + " was in songs, but not in df_languages.")

# merge df_languages and language on Country and Country/Region
songs = pd.merge(songs, df_languages[['Country_code2', 'Official_languages']], left_on='Country_code2', right_on='Country_code2', how='left')


In [ ]:
# Tidy the languages column
songs['Official_languages'] = songs['Official_languages'].fillna(' ')

# Add more columns
def get_n_languages(langs):
    """Get the number of languages in a list of languages"""
    if '6 other' in langs:   # ["english", "6 other"] -> 7
        return len(langs) + 5
    elif '10 other' in langs:
        return len(langs) + 9
    else:
        return len(langs)
songs['Contains_Multiple_Languages'] = songs['Language_sung'].apply(lambda x: len(x) > 1)
songs['Number_of_Languages'] = songs['Language_sung'].apply(get_n_languages)
songs['Contains_Own_Language'] = songs.apply(lambda df: len(set(df['Language_sung']).intersection(df['Official_languages'].split())) > 0, axis=1)

songs[songs['Number_of_Languages'] > 3]

In [ ]:
# determine whether the song is performed in the language of the voting country
df_voting_language = df_languages.copy()
df_voting_language['Voting_Languages'] = df_voting_language['Official_languages']

votes = pd.merge(votes, df_voting_language[['Country_code2', 'Voting_Languages']], 
                 left_on='from_code2', right_on='Country_code2', how='left')

# Combine votes and language
df_VL = pd.merge(votes, songs, left_on=['to_code2', 'year'], right_on=['Country_code2', 'Year'], how='left')
df_VL['Contains_Voting_Language'] = df_VL.apply(lambda df: len(set(df['Language_sung']).intersection(df['Voting_Languages'].split())) > 0, axis=1)

df_VL.head()

In [ ]:
# check if Country and to_country are identical
if not all([all(df_VL['Country'] == df_VL['to_country']),
            all(df_VL['Country_code2_x'] == df_VL['from_code2']),
            all(df_VL['Country_code2_y'] == df_VL['to_code2']),
            all(df_VL['Year'] == df_VL['year'])]):
    raise ValueError("Mismatch in the merge - check this out!")

df_VL = df_VL[[
    'year', 'Artist',
    'from_country',	'to_country', 'points', 'total_points', 
    'rank',	'from_code2', 'from_code3', 'to_code2', 'to_code3',
    'Official_languages', 'Language_sung',
    'Contains_English', 'Contains_NonEnglish', 'Contains_Multiple_Languages',
    'Number_of_Languages', 'Contains_Own_Language', 'Contains_Voting_Language']]

df_VL.shape

In [ ]:
df_VL.to_csv('data/language.csv', index=False)

# Performer gender

We determine the artists gender by scraping the wikipedia page for the performing artist. Artists are classed as either male, female or group. 

Note that this classification is currently binary. We are aware that some artists do not perform as male or female (e.g. Conchita Wurst) therefore this is not a completely accurate representation. 


In [ ]:
async def get_property(session, concept_id, property_id):
    """Async reimplementation of wikipeople.get_property
    https://github.com/samvanstroud/wikipeople/blob/master/wikipeople/wikipeople.py
    
    session is an aiohttp ClientSession.
    concept_id can be obtained using the get_concept_id function
    property_id is hardcoded, I don't know where to get these from, but whatever.
    
    Returns None if any of this can't be found for whatever reason.
    
    e.g. "Q219655" is the concept_id for Carey Mulligan; "P21" is the property_id for gender. So we have that
        get_property(session, "Q219655", "P21") -> "female"
    """
    url = 'https://www.wikidata.org/w/api.php'
    params = {'action': 'wbgetclaims',
              'entity': concept_id,
              'property': property_id,
              'language': 'en',
              'format': 'json'}
    async with session.get(url, params=params) as resp:
        try:
            res = await resp.json()
        except Exception as e:
            print(resp)
            raise e

    if property_id not in res['claims']:
        return None
    # This gives yet another 'id', and we then need to perform yet another HTTP
    # request to find the actual *label* that this corresponds to.
    else:
        id = None
        for prop in res['claims'][property_id]:
            try:
                id = prop['mainsnak']['datavalue']['value']['id']
            except:
                continue

        if id is None:
            return None
        else:
            new_params =  {'action': 'wbgetentities',
                           'ids': id,
                           'languages': 'en',
                           'format': 'json',
                           'props': 'labels'}
            async with session.get(url, params=new_params) as resp:
                try:
                    res = await resp.json()
                except Exception as e:
                    print(resp)
                    raise e
            try:
                return res['entities'][id]['labels']['en']['value']
            except:
                return None

async def get_concept_id(session, page_name):
    """
    Get the concept_id corresponding to a particular Wikipedia page. For some odd reason, some Wikipedia
    pages don't have concept IDs. In such a case, we return None.
    
    e.g. get_concept_id(session, "Carey Mulligan") -> "Q219655"
    """
    url = 'https://www.wikidata.org/w/api.php'
    params = {'action': 'wbsearchentities',
              'search': page_name,
              'language': 'en',
              'format': 'json'}
    music_markers = [
        'singer', 'artist', 'musician', 'music',
        'band', 'group', 'duo', 'ensemble'
    ]

    async with session.get(url, params=params) as resp:
        # Titles of WP pages that match the search query.
        json = await resp.json()

    result = json['search']

    if len(result) == 0:
        # Couldn't find a concept id for the person/group
        return None

    # By default, choose the first result from the list
    target = 0
    # But check the other results to see if any of them are musicians (as
    # indicated by the markers) and Eurovision contestants
    for i, res in enumerate(result):
        if 'description' in res['display']:
            description = res['display']['description']['value']
            if any(markers in description for markers in music_markers):
                concept_id = res['id']
                contestant_in = await get_property(session, concept_id, 'P1344')
                if contestant_in is not None and "Eurovision" in contestant_in:
                    target = i
    # Return the concept ID of the result found
    return result[target]['id']

async def lookup_gender(session, page_name):
    """Find gender of a performing act, using the name associated with their
    Wikipedia page. Returns None if could not be found.
    """
    concept_id = await get_concept_id(session, page_name)
    if concept_id is None:
        return None

    gender = await get_property(session, concept_id, 'P21')
    instance = await get_property(session, concept_id, 'P31')
    if gender is None and instance is None:
        return None
    elif gender is None and instance is not None:
        group_checks = ["group", "duo", "trio", "music", "band", "ensemble"]
        if any(x in instance for x in group_checks):
            return "group"
    else:
        return gender

async def get_pages(session, name):
    """Obtain a list of Wikipedia pages obtained by searching for a name.
    """
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "opensearch",
        "namespace": "0",
        "search": name,
        "limit": "10000",
        "format": "json"
    }
    async with session.get(url, params=params) as resp:
        # Titles of WP pages that match the search query.
        json = await resp.json()
    return json[1]

async def get_artist_gender(session, name):
    gender = None
    # Get the WP page for this person/group
    pages = await get_pages(session, name)
    # If there's one, try to get their gender from the first page
    if len(pages) > 0:
        gender = await lookup_gender(session, pages[0])
    # Finally we use some heuristics to cover some edge cases
    if gender is None:
        if '&' in name or 'feat.' in name:
            return 'group'
    
    return gender

In [ ]:
# Check whether the gender data has already been saved. If so, load it in.
p = Path("data/gender_dict.json")
if p.is_file():
    with open(p, 'r') as file:
        gender_dict = json.load(file)
        print('Loaded performer genders from file')
        
else:
    # If not, now that we have all the necessary functionality, we can fetch the data from Wikipedia.
    all_performers = df_VL['Artist'].unique().tolist()
    n_performers = len(all_performers)
    MAX_CONCURRENT = 40   # To stop Wikipedia from complaining about 'too many requests'
    USER_AGENT = 'Eurovision study @ The Alan Turing Institute mailto:jyong@turing.ac.uk'

    async def get_all_genders():
        genders = []
        print(f'We need to fetch the genders of {n_performers} performers, in batches of {MAX_CONCURRENT}. Hold tight...')
        async with aiohttp.ClientSession(headers={'User-Agent': USER_AGENT}) as session:
            start = 0
            end = MAX_CONCURRENT
            while start < n_performers:
                print(f'Getting genders for performers #{start + 1} to #{end}... ', end='')
                batch_tasks = asyncio.gather(*[get_artist_gender(session, p) for p in all_performers[start:end]])
                batch_genders = await batch_tasks
                print(f'Got {len(batch_genders)} results, {len([g for g in batch_genders if g is None])} of which were None.')
                genders = genders + batch_genders
                start = end
                end = min(end + MAX_CONCURRENT, n_performers)
                await asyncio.sleep(1.5)   # Put a pause between batches to avoid being timed out
        # now pray that I didn't make an off-by-one error somewhere
        assert len(genders) == n_performers
        print('Finished downloading gender data.')
        return dict(zip(all_performers, genders))
        
    gender_dict = await get_all_genders()
    
    # Manually assign missing entries (the Nones).
    male = ['Michael Hajiyanni', 'Charlie', 'Tüzmen', 'Mietek Szcześniak', 'Olexandr', 'Max', 'Brinck',
            'Sakis Rouvas (2)', 'Gianluca', 'Frans', 'Chingiz', 'Mahmood', 'Serhat (2)', 'Miki', 'Stefan']
    female = ['Gunvor', 'Selma', 'Charlotte Nilsson (Perrelli)', 'Karolina', 'Laura', 'Rosa', 'Lou', 'Nicola',
            'Karmen', 'Sanda', 'Ortal', 'Gracia', 'Chiara (2)', 'Hanna', 'Chiara (3)', 'Elena', 'Lena (2)',
            'Birgit', 'Samra', 'ZAA Sanja Vučić', 'Anja', 'Alma', 'Netta', 'Michela', 'Efendi', 'Victoria',
            'Destiny', 'Amanda Georgiadi Tenfjord', 'MARO']
    group = ['Eden', 'Voice', 'Taxi', 'One', 'Prime Minister', 'Fame', 'Regina (band)', 'ESDM',
            'Tolmachevy Sisters', 'Minus One', 'AWS']
    for p in male:
        gender_dict[p] = "male"
    for p in female:
        gender_dict[p] = "female"
    for p in group:
        gender_dict[p] = "group"

    # Wikipedia needs to learn that 'trans woman' is 'female'.
    for k, v in gender_dict.items():
        if v == 'trans woman':
            gender_dict[k] = 'female'
            
    # Save it to a file
    with open('data/gender_dict.json', 'w') as file:
        json.dump(gender_dict, file)

In [ ]:
# Add gender to the dataframe.
df_VLG = df_VL.copy()
df_VLG['gender'] = df_VLG['Artist'].map(gender_dict)
df_VLG.head()

In [ ]:
check_consistency(df_VLG)

In [ ]:
df_VLG.to_csv('data/gender.csv', index=False)

# Migration

- The `migration-flows.csv` data is from [Our World in Data](https://ourworldindata.org/migration) on international migration, under the 'Explore data on where people migrate from and to' section.
- Original source is from the UN.
- Data shows total number of immigrants in each country split by country of origin in the years 1990-2020, recorded at intervals of every 5 years.
- Additional population size data (`pop_sizes.csv`) is taken from the [World Bank](https://data.worldbank.org/indicator/SP.POP.TOTL?end=2021&start=2021&view=map).

In [ ]:
migration = pd.read_csv('data/migration-flows.csv')

# Martin actually writes good pandas code, unlike me

migration = (migration
    .pipe(pd.melt, id_vars=['Country', 'Year'], var_name='Migration', value_name='Count')  # to long format
    .loc[lambda x: x['Migration'].str.contains('Emigrants')]                               # filter for emigrant rows
    .pipe(lambda x: x.rename(columns = {col: col.lower() for col in x.columns}))           # lowercase column names                                                         
    .assign(migration = lambda x: x.migration.str.replace('Emigrants from ', ''))          # filter for emigrant rows                          
    .rename(columns={'migration': 'emigrated_from', 'country': 'emigrated_to'})            # boil down to country name
    .query('count >= 0')                                                                   # negative counts are just total emigrants from country
    .pipe(lambda x: x.assign(count = x['count'].astype(int)))                              # convert count to int     
)
migration.head()

In [ ]:
# Clean up country names
for ft in ['from', 'to']:
    migration[f'emigrated_{ft}'] = migration[f'emigrated_{ft}'].str.lower()
    migration.loc[migration[f'emigrated_{ft}'] == 'moldova', f'emigrated_{ft}'] = 'moldova, republic of'
    migration.loc[migration[f'emigrated_{ft}'] == 'russia', f'emigrated_{ft}'] = 'russian federation'

# Remove countries we don't care about
ev_countries = set(df_VLG['from_country'].unique()).union(set(df_VLG['to_country'].unique()))
migration = migration[(migration['emigrated_to'].isin(ev_countries)) & (migration['emigrated_from'].isin(ev_countries))]

migration_countries = set(migration['emigrated_to'].unique()).union(set(migration['emigrated_from'].unique()))
print(ev_countries - migration_countries)  # No data for Yugoslavia.

# Add in country codes
for ft in ['from', 'to']:
    migration[f'emigrated_{ft}_code2'], migration[f'emigrated_{ft}_code3'] = zip(*migration[f'emigrated_{ft}'].map(get_country_codes))
    
migration = migration.reset_index(drop=True)
migration.head()

In [ ]:
pop_size = (pd.read_csv('data/pop_sizes.csv')
           .iloc[:, 3:]
           .rename(columns=lambda x: x.lower().replace(' ', '_'))
           .pipe(pd.melt, id_vars=['country_code'], var_name='year', value_name='population')
           .assign(year=lambda x: x['year'].apply(lambda y: y.split('_')[0]))
           .assign(year=lambda x: x['year'].astype(int))
           .rename(columns={'country_code': 'code3'})
           .dropna()
           .assign(population=lambda x: pd.to_numeric(x['population'], errors='coerce'))
)
pop_size.head()

migration_and_pop = (migration.merge(pop_size, left_on=['year', 'emigrated_to_code3'], right_on=['year', 'code3'], how='left')
                     .rename(columns={'population': 'population_to'})
                    .assign(prop_emigrants=lambda x: x['count'] / x['population_to'])
                    #.reindex(columns=['country', 'code', 'code3', 'population', 'year', 'emigrated_from_code', 'count', 'prop_emigrants'])
                    )
migration_and_pop.head(n=20)

Because we don't have migration data for every year, when merging with the main dataset, we take the last migration data point before the competition.
So, for example, the 2012 entries will contain migration data from 2010.

To do this, we'll first make 5 copies of each row from the `migration_and_pop` dataframe, each with a different year.

In [ ]:
migration_and_pop['migration_pop_year'] = migration_and_pop['year']

total_migration_and_pop = migration_and_pop.copy()
for i in range(1, 5):
    next_migration_and_pop = migration_and_pop.copy()
    next_migration_and_pop['year'] = next_migration_and_pop['year'] + i
    total_migration_and_pop = pd.concat([total_migration_and_pop, next_migration_and_pop], ignore_index=True)
    
total_migration_and_pop = total_migration_and_pop.sort_values(by=["emigrated_from", "emigrated_to", "year"])

total_migration_and_pop = total_migration_and_pop[['emigrated_from_code2', 'emigrated_to_code2', 'year', 'count', 'population_to', 'prop_emigrants', 'migration_pop_year']]

total_migration_and_pop.head(n=20)

In [ ]:
# Now we can join with the main dataframe.

# migration_v2p      -> number of migrants from voting country to performing country
# population_p       -> population of performing country
# prop_emigrants_v2p -> proportion of migrants from voting country in population of performing country
df_VLGM = df_VLG.merge(total_migration_and_pop, how='left', left_on=['from_code2', 'to_code2', 'year'], right_on=['emigrated_from_code2', 'emigrated_to_code2', 'year'])
df_VLGM = (df_VLGM
           .drop(columns=['emigrated_from_code2', 'emigrated_to_code2', 'migration_pop_year'])
           .rename(columns={'count': 'migration_v2p', 'population_to': 'population_p', 'prop_emigrants': 'prop_emigrants_v2p'})
)

# migration_p2v      -> number of migrants from performing country to voting country
# population_p       -> population of voting country
# prop_emigrants_v2p -> proportion of migrants from performing country in population of voting country
# migration_pop_year -> year from which the migration and population data is taken
df_VLGM = df_VLGM.merge(total_migration_and_pop, how='left', left_on=['from_code2', 'to_code2', 'year'], right_on=['emigrated_to_code2', 'emigrated_from_code2', 'year'])
df_VLGM = (df_VLGM
           .drop(columns=['emigrated_from_code2', 'emigrated_to_code2'])
           .rename(columns={'count': 'migration_p2v', 'population_to': 'population_v', 'prop_emigrants': 'prop_emigrants_p2v'})
)
df_VLGM['migration_pop_year'] = df_VLGM['migration_pop_year'].astype(int, errors='ignore')   # ignore NaN's.

df_VLGM.head(n=10)

In [ ]:
df_VLGM.to_csv('data/migration.csv', index=False)

# Comps without win

Copy the winners from wikipedia. Note that Luxembourg withdrew from the contest in 1994, so is not included in our data - hence why it does not merge, we will remove this from the data. 

In [ ]:
url = (
    f"https://en.wikipedia.org/wiki/List_of_Eurovision_Song_Contest_winners"
)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find_all("table", {"class": "wikitable"})[0]
winners = pd.DataFrame(pd.read_html(str(table))[0])

winners = winners.loc[winners['Year'] != 2020, ['Year', 'Country']]
winners = list(winners.to_records(index=False))

winners = [(year, 'Russian federation' if Country == 'Russia' else Country) for year, Country in winners]
winners = [(year, get_country_codes(Country.lower())[0]) for year, Country in winners if Country != 'Luxembourg']

In [ ]:
# Construct a dictionary mapping each country to the years they won in.
all_wins = {}

# for each row in winners, get the country code and year
for y, code in winners:
    if code in all_wins:
        all_wins[code].append(y)
    else:
        all_wins[code] = [y]

print(all_wins)

In [ ]:
df_VLGMC = df_VLGM.copy()

def comps_without_win(code, year):
      # Find last win. Use 1955 (year before ESC started) if there isn't one.
      if code not in all_wins:
        last_win = 1955
      else:
        last_win = max([y for y in all_wins[code] if y < year], default=1955)
      
      # Count the number of competitions since the last win. Note that the 2020
      # contest was cancelled.
      comps = year - last_win - 1
      if year > 2020 and last_win < 2020:
        comps = comps - 1

      return comps

# Some quick tests.
assert(comps_without_win("UA", 2023) == 0)   # won in 2022
assert(comps_without_win("GB", 2023) == 24)  # won in 1997
assert(comps_without_win("AU", 2023) == 66)  # never won
assert(comps_without_win("SE", 1983) == 8)   # won in 1974
assert(comps_without_win("SE", 2019) == 3)   # won in 2015
assert(comps_without_win("NL", 2019) == 43)  # won in 1975

df_VLGMC['comps_without_win'] = df_VLGMC.apply(lambda row: comps_without_win(row['to_code2'], row['year']), axis=1)

In [ ]:
df_VLGMC.tail()

In [ ]:
df_VLGMC.to_csv('data/comps_without_win.csv', index=False)

# Border data

Raw data is obtained from GeoDataSource: https://github.com/geodatasource/country-borders/

In [ ]:
border = pd.read_csv('data/geodatasource-country-borders.csv')
border.head()

In [ ]:
# Clean up the data first; subset to only Eurovision countries
ev_code2s = set(df_VLGMC['from_code2'].unique()).union(set(df_VLGMC['to_code2'].unique()))
border = border[(border['country_code'].isin(ev_code2s)) & (border['country_border_code'].isin(ev_code2s))]

# Generate a list of tuples
border_tuples = list(border[['country_code', 'country_border_code']].itertuples(index=False, name=None))
# Sanity check to make sure the list is symmetric. Expect True.
all((b, a) in border_tuples for a, b in border_tuples)

# Yugoslavia needs a manual exception. For now, Yugoslavia shares a border with 
# country X if X shares a border with either Serbia or Montenegro.
def has_border(cty1, cty2):
    if cty1 == 'YU':
        return has_border('RS', cty2) or has_border('ME', cty2)
    elif cty2 == 'YU':
        return has_border(cty1, 'RS') or has_border(cty1, 'ME')
    else:
        return (cty1, cty2) in border_tuples
# TODO: CHECK IF THIS IS HISTORICALLY CORRECT

In [ ]:
has_border('BA', 'YU')

In [ ]:
# Then just add a new column to df_VLGMC that is True if the two countries are neighbours.

df_VLGMCB = df_VLGMC.copy()
df_VLGMCB["has_border"] = df_VLGMCB.apply(lambda row: has_border(row['from_code2'], row['to_code2']), axis=1)
df_VLGMCB.head(n=20)

Store the data as CSV

In [ ]:
# Add a row to represent the competition round the votes apply to (which for historical data, is always the final)
df_VLGMCB['comp_round'] = 'f'

df_VLGMCB.to_csv('data/df_main.csv', index=False)

# Add the 2023 Performance data 


This will be used for the purposes of predictions

In [ ]:
class GenderGuess():
    """Guess the gender of a performer based on entries from Wikidata.
    """
    future = False
    exceptions = {"Brunette": "female"}

    def __init__(self, future=False):
        print("Initialising gender guesser")
        pass

    def __search_wikidata(self, string):
        """
        Query the Wikidata API using the wbsearchentities function.
        Return the concept ID of the search result that has the musician identifier.
        """
        query = 'https://www.wikidata.org/w/api.php?action=wbsearchentities&search='
        query += string
        query += '&language=en&format=json'
        music_markers = [
            'singer', 'artist', 'musician', 'music',
            'band', 'group', 'duo', 'ensemble'
        ]
        res = requests.get(query).json()
        if len(res['search']) == 0:
            return None

        target = 0
        for i in range(len(res['search'])):
            if 'description' in res['search'][i]['display']:
                description = res['search'][i]['display']['description']['value']
                if any(markers in description.lower() for markers in music_markers):
                    if self.future:
                        target = i
                        break
                    else:
                        concept_id = res['search'][i]['id']
                        contestant_in = wp.get_property(concept_id, 'P1344')[-1]
                        if "Eurovision" in contestant_in:
                            target = i
                            break

        return res['search'][target]['id']

    def __lookup_gender(self, name):
        """Find gender of given name. If the name is not related to a wiki entry 
        it will return 'RNF' (record not found). Alternatively it will return 
        the gender if the record has one or NA if it does not have this 
        property.
        Args:
            name (str): The name to search
        Returns:
            str: The gender of the person searched
        """
        gender = 'RNF'
        data = self.__search_wikidata(name)
        if data:
            gender = wp.get_property(data, 'P21')[-1]
            instance = wp.get_property(data, 'P31')[-1]
            if gender == 'NA':
                group_checks = [
                    "group", "duo", "trio", "music", "band", "ensemble"
                ]
                if any(x in instance for x in group_checks):
                    gender = "group"
        return gender

    def __get_artist_gender(self, search):
        if search in self.exceptions:
            return self.exceptions[search]

        s = requests.Session()
        url = "https://en.wikipedia.org/w/api.php"
        params = {
            "action": "opensearch",
            "namespace": "0",
            "search": search,
            "limit": "10000",
            "format": "json"
        }
        r = s.get(url=url, params=params)
        names = r.json()[1]
        gender = ""
        if len(names) < 1:
            gender = "RNF"
        else:
            for n in names:
                gender = self.__lookup_gender(n)
                if not any(gender == x for x in ["RNF", "NA"]):
                    return gender
        if gender == "RNF" or gender == "NA":
            if ("&" in search) or (' and ' in search):
                gender = "group"
        return gender

    def guess_gender(self, artist):
        """Guess the gender of an artist.
        Returns a string representing the gender of the artist.
        Args:
            artist (str): The name of the artist
        Returns:
            str: One of "group", "female", "male" or "RNF"
        """
        gender = self.__get_artist_gender(artist)
        #print("Artist: {}, gender: {}".format(artist, gender))
        return gender

In [ ]:
df_VLGMCB['from_country'].unique()

In [ ]:
import pandas as pd
import numpy as np
import requests
import sys
from pathlib import Path
from bs4 import BeautifulSoup
from wikipeople import wikipeople as wp # use wp to get the gender data as needed.

class EurovisionFuture():
    """Collect together information about acts that might appear in the
    Eurovision final, based on their appearance in the semi finals.
    """
    in_file = None
    out_file = None
    countries = None
    result = None
    language_map = None
    df = None
    language_match = None
    gender = None

    def __init__(self, in_file, out_file):
        self.in_file = in_file
        self.out_file = out_file

        # Don't run if the files aren't present
        for file in [in_file,]:
            if not file.is_file():
                raise FileNotFoundError(f"file {str(file)} was not found")

        #self.__load_country_codes()
        self.__import_existing()
        #self.language_match = LanguageMatch(self.country_codes_dict)
        self.gender = GenderGuess(future=True)

    def __expand_data(self, collected):
        """Expand the data using a cross join on voting countries.
        For each row in the data frame, generate a duplicate for each country
        that might vote for it, filling the "From country" column with the
        voting country.
        Args:
            collected (pd.DataFrame): the current data to be expanded
        Returns:
            pd.DataFrame: the existing data expanded with the "From country" column
        """
        from_country = pd.DataFrame({"from_country": df_VLGMCB['from_country'].unique()})
        collected = pd.merge(collected, from_country, how="cross")

        collected['from_code2'], collected['from_code3'] = zip(*collected['from_country'].map(get_country_codes))

        # Remove rows where country and from_country are the same (self-voting)
        collected = collected.drop(collected[collected["to_code2"] == collected["from_code2"]].index)

        # Reset the index
        collected = collected.reset_index(drop=True)


        return collected

    def __transfer_existing_data(self, collected):
        """Transfer data from the historical data frame into the future frame.
        Transfers border and migration data from the most recent year for which
        it exists in the historical data into the future data frame.
        Args:
            collected (pd.DataFrame): the current data to be augmented
        Returns:
            pd.DataFrame: data updated with border and migration details
        """
        # Fill out the has_border entries from existing data
        print('Transfering border data')
        collected['has_border'] = collected[['to_code2', 'from_code2']].apply(lambda x: self.__get_has_border(x['to_code2'], x['from_code2']), axis=1)

        # Fill the voting language data
        collected = pd.merge(collected, df_voting_language[['Country_code2', 'Voting_Languages']], left_on='from_code2', right_on='Country_code2', how='left')
        collected['Contains_Voting_Language'] = collected.apply(lambda df: len(set(df['Language_sung']).intersection(df['Voting_Languages'].split())) > 0, axis=1)

        # Fill out migration data
        print('Transfering migration data')
        migration_pop_cols = ['migration_v2p', 'prop_emigrants_v2p', 'migration_p2v',
       'population_v', 'prop_emigrants_p2v', 'migration_pop_year']

        collected[migration_pop_cols] = collected[['to_code2', 'from_code2']].apply(lambda x: self.__get_migration(x['to_code2'], x['from_code2']), axis=1)
        
        collected['points'] = np.nan
        collected['total_points'] = np.nan
        collected['rank'] = np.nan

        print(collected['to_code2'].unique())

        return collected

    def __get_has_border(self, code, from_country):
        """Check whether there's a land border bertween two countries
        Args:
            code (str): the country code of one country
            from_country (str): the country code of another country
        Returns:
            iot: True if the countries share a land border, False otherwise
        """
        result = self.df[(self.df['to_code2'] == code) & (self.df['from_code2'] == from_country)].nlargest(1, 'year')['has_border'].values
        return result[0] if len(result) > 0 else False

    def __get_migration(self, code, from_country):
        """Get migration data from one countrhy to another
        Args:
            code (str): the country code for migrating to
            from_country (str): the country code for migraition from
        Returns:
            [band, year, count, prop]: a list containing respectively the migration band (year), the year the data relates to
                                       the number of migratnts and the proportion of population of the country migrating from
        """


        migration_pop_cols = ['migration_v2p', 'prop_emigrants_v2p', 'migration_p2v',
       'population_v', 'prop_emigrants_p2v', 'migration_pop_year']

        result = self.df[(self.df['to_code2'] == code) & (self.df['from_code2'] == from_country)].nlargest(1, 'year')[migration_pop_cols].values
        result = result[0] if len(result) > 0 else [0] * len(migration_pop_cols)
        return pd.Series(result)#, index=[migration_pop_cols])

    def __get_population(self, code):
        """Get the most recent population info for a country
        Args:
            code (str): the country code for the country to check
        Returns:
            int: population data for the country for the most recent year found in the historical data frame
        """
        result = self.df[self.df['to_code2'] == code][['year', 'population_p']].nlargest(1, 'year')['population_p'].values
        return result[0] if len(result) > 0 else 0
    
    def process(self, explode):
        """Process the data
        Downloads and processes the data for the semi finals in order to 
        generate data for the finals.
        
        If the explode parameter is set to True, the output will contain multiple rows for each
        country, one for each voting country. If set to False, each country will have only a
        single row.
        
        Args:
            explode (bool): set to True to multiply the rows up by voting countries, False otherwise
        Returns:
            None
        """

        # Get the performance data
        semi_final_1 = self.__import_participants_from_wiki(2023, 'Semi-final_1', 'sf1')
        semi_final_2 = self.__import_participants_from_wiki(2023, 'Semi-final_2', 'sf2')
        semi_final_3 = self.__import_participants_from_wiki(2023, 'Final', 'f')
        semi_finals = pd.concat([semi_final_1, semi_final_2, semi_final_3])

        # Populate the vote-performance data
        if explode:
            semi_finals = self.__expand_data(semi_finals)
            semi_finals = self.__transfer_existing_data(semi_finals)
        else:
            # Clear the columns that don't make sense without a "from_country" voter
            for column in ['from_country', 'points', 'total_points', 'rank', 'from_code2', 'from_code3', 
                           'migration_v2p', 'prop_emigrants_v2p', 'migration_p2v', 'has_border',
                           'migration_pop_year', 'prop_emigrants_p2v', 'population_v']:
                semi_finals[column] = np.nan

        # if any df_VLGMCB columns are not in semi-finals, raise an exception
        if len(set(df_VLGMCB.columns) - set(semi_finals.columns)) > 0:
            raise Exception("Missing columns in semi_finals:", set(df_VLGMCB.columns) - set(semi_finals.columns))

        # Match columns
        semi_finals = semi_finals[df_VLGMCB.columns]

        # Store the result
        self.result = semi_finals

    def get_result(self):
        """Get the result of all that processing
        As long as processing has been performed, this will return a data frame of data for the
        acts that are likely to appear in the Eurovision final based on their appearance in the
        semi finals.
        Args:
            None
        Returns:
            pd.DataFrame: the processed data
        """
        return self.result

    def __import_existing(self):
        """
        Import the 1998-2022 data set. We need it later do copy some data into the 2023 rows.
        Args:
            None
        Returns:
            None
        """
        # Read in CSV
        self.df = pd.read_csv(self.in_file)
        self.df['has_border'] = self.df['has_border'].fillna(0)

        self.df = self.df.reset_index(drop=True)

    def __last_win(self, country, year):
        """Returns the number of years since the last win
        Returns the number of years since the last win based on the historical data.
        This does not get updated for the most recent winner.
        Args:
            country (str): the country code for the country to get the info for
            year (int): the current year
        Returns:
            int: the number of years since the country last won
        """
        result = self.df[self.df['to_code2'] == country][['year', 'comps_without_win']].nlargest(1, 'year').values
        recent_year, duration = result[0] if len(result) > 0 else [0, 0] 
        # We're going to assume year > recent_year to make our lives easier
        assert(year > recent_year)
        return duration + (year - recent_year)

    def __import_participants_from_wiki(self, year, header_id, comp_round):
        """Download the data from the wiki for a particular semi final
        Downloads artist data from Wikipedia based on the year and the subheading anchor.
        Data is scraped from the first table in the section with the anchor tag or id provided.
        Args:
            year (int): the year to get the data for
            header_id (str): the anchor for the section to scrapte the table data from
        """
        url=f"https://en.wikipedia.org/wiki/Eurovision_Song_Contest_{year}"
        print('Downloading wikipedia page: {}'.format(url))
        response=requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        tables = soup.find_all('table',{'class':"wikitable"})

        table = soup.find(id=header_id).find_all_next('table')[0]

        df_table=pd.read_html(str(table))
        df_table=pd.DataFrame(df_table[0])

        # Remove citations references from the column titles
        df_table.columns = [x.split('[')[0] for x in df_table.columns]

        # These values are the same for every row
        df_table['year'] = year

        # Country codes
        df_table['to_country'] = df_table['Country'].str.lower()
        df_table['to_country'] = df_table['to_country'].map(standardise_country)
        df_table['to_code2'], df_table['to_code3'] = zip(*df_table['to_country'].map(get_country_codes))

        # Convert the entry to a list of languages and strip any citation references
        print('Deriving language entries')
        df_table['Language_sung'] = df_table['Language(s)'].apply(lambda x: [y.lower().split('[')[0] for y in x.split(', ')])
        df_table['Contains_English'] = df_table['Language_sung'].apply(lambda x: 'english' in x)
        df_table['Contains_NonEnglish'] = df_table['Language_sung'].apply(lambda x: x != ['english'])

        # print any countries in songs['Country'] that are not in df_languages['Country/Region']
        if len(set(songs['Country_code2']) - set(df_languages['Country_code2'])) > 0: 
            countries = list(set(songs['Country_code2']) - set(df_languages['Country_code2']))
            raise KeyError("Country name " + ', '.join(countries) + " was in songs, but not in df_languages.")

        # merge df_languages and language on Country and Country/Region
        df_table = pd.merge(df_table, df_languages[['Country_code2', 'Official_languages']], left_on='to_code2', right_on='Country_code2', how='left')
        
        # Tidy the languages column
        df_table['Official_languages'] = df_table['Official_languages'].fillna(' ')
        df_table['Contains_Multiple_Languages'] = df_table['Language_sung'].apply(lambda x: len(x) > 1)
        df_table['Number_of_Languages'] = df_table['Language_sung'].apply(get_n_languages)
        df_table['Contains_Own_Language'] = df_table.apply(lambda df: len(set(df['Language_sung']).intersection(df['Official_languages'].split())) > 0, axis=1)


        # Figure out the gender from Wikipedia
        print('Guessing genders')
        df_table['gender'] = df_table['Artist'].apply(lambda x: self.gender.guess_gender(x))

        # Copy over data from the existing dataset
        print('Transfering population data')
        df_table['population_p'] = df_table['to_code2'].apply(lambda x: self.__get_population(x))

        print('Calculating last win')
        df_table['comps_without_win'] = df_table['to_code2'].apply(lambda x: self.__last_win(x, year))
        
        # Add in the competition round data
        df_table['comp_round'] = comp_round

        # Add in the competition round data
        df_table['comp_round'] = comp_round

        return df_table

    def save(self, out_file):
        """Save the result to a CSV file
        """
        print("Writing out results to: {}".format(out_file))
        self.result.to_csv(out_file)

def print_syntax():
	print('Syntax: get_future_performers.py <input-file> <country-pickle> <out-file>')
	print()
	print('\tCollect data about future Eurovision performers')
	print('\t<input-file>     : CSV file containing data for previous years')
	print('\t<country-pickle> : pickle file mapping countries to country codes')
	print('\t<out-file>       : file to save the output CSV to')
	print()
	print('Example usage')
	print('\tget_future_performers.py eurovision_merged_covariates_03Feb.csv country_codes_dict.pickle out.csv')

In [ ]:
in_file = Path("data/df_main.csv")
out_file = Path("data/eurovision_2023.csv")

print("Historical data CSV: {}".format(in_file))
print("Out file: {}".format(out_file))

# # Initalise
future = EurovisionFuture(in_file, out_file)

Next cell takes around 2 mins to run

In [ ]:
future.process(False)

In [ ]:
future.result

In [ ]:
future.result['to_country'].value_counts()

In [ ]:
future.result.to_csv('data/df_2023.csv', index=False)